In [284]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm
import psycopg2
from sqlalchemy import create_engine
import sys

In [ ]:
#You will need to store your db credentials in a file in a dictionary named "creds"
exec(open('./db.cred').read())

In [270]:
engine = create_engine('postgresql://{user}:{password}@{host}/{dbname}', echo=False)

In [178]:
conn = psycopg2.connect("dbname='{dbname}' user='{user}' host='{host}' password='{password}'")

In [3]:
local_news['file_name'] = local_news['name'].map(lambda x: x.lower().replace(' ', '_') + '.html')

In [4]:
def getRss(file_name):
    try:
        with open('./front_pages/%s' % file_name) as f:
            bs = BeautifulSoup(f, 'html.parser')
            return bs.find_all('link', {'type': 'application/rss+xml'})[0]['href']
    except:
        pass

In [48]:
#local_news[~local_news.rss_link.isnull()].to_pickle('rss_links.pkl')
# Some code--getting the rss links from the local copies of their html homepages and updating it in the db--got lost here
# but it's all in the local_news table

In [51]:
# just the ones with rss links
local_news = local_news[~local_news.rss_link.isnull()]

In [386]:
# Make all hashes postitive
def posHash(x):
    h = hash(x)
    if h < 0:
          return h + sys.maxsize
    return h

def getArticles(row):
    # this is a bug many sites seem to have--including this line prevents any news articles from being included
    rss_link = row['rss_link'].replace('&k[]=%23topstory', '')
    r = requests.get(rss_link)
    bs = BeautifulSoup(r.text, 'html5lib')
    articles = pd.DataFrame([(item.find('title').text, 
      item.find('pubdate').text,
      item.find('description').text,
      item.find('link').next_element.strip(),
     )
    for item in bs.find_all('item')], columns=['headline', 'pub_date', 'description', 'link'])
    
    articles['pub_date'] = pd.to_datetime(articles['pub_date'])
    articles['pub_name'] = row['name']
    articles['pub_id'] = row['id']
    
    # the primary id for the rss_articles table is the positive 
    # hash of the concatinated pub_name and headline
    articles['id'] = articles.apply(lambda x: posHash(x['pub_name'] + x['headline']), axis=1)
    
    # drop duplicates
    return articles[~articles.duplicated('id', keep=False)]

In [345]:
# I have no idea how duplicates got in the local news table... todo: fix this
rss_news = local_news[local_news.rss_link != 'None'].drop_duplicates('name')

In [ ]:
# Some of the rss links don't include the base url
def addBaseUrl(row):
    if notrow['url'] in row['rss_link']:
            return row['url']
    return row['url'] + row['rss_link']

In [362]:
rss_news['rss_link'] = rss_news.apply(lambda x: x['rss_link'] if 'http' in x['rss_link'] else x['url'] + x['rss_link'], axis=1)

In [423]:
logfile = open('pull_rss.log', 'w')
errorfile = open('pull_rss.error', 'w')

In [431]:
cur.execute("DROP TABLE IF EXISTS rss_articles")
conn.commit()

for i, row in tqdm(enumerate(rss_news.iterrows())):
    try:
        articles = getArticles(row[1])
        if i == 0:
            articles.to_sql('rss_articles', engine, if_exists='replace', index=False)
            cur.execute("ALTER TABLE rss_articles ADD PRIMARY KEY (id)")
            conn.commit()
        else:
            articles.to_sql('rss_articles', engine, if_exists='append', index=False)
        logfile.write("Pulled %d articles for %s\n" % (len(articles), row[1]['name']))
        logfile.flush()
    except:
        errorfile.write("Error pulling articles from %s\n" % row[1]['rss_link'])
        errorfile.flush()

176it [00:59,  2.94it/s]
